In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00235/household_power_consumption.zip
!unzip household_power_consumption.zip

In [143]:
# The ds (datestamp) column should be of a format expected by Pandas,
# ideally YYYY-MM-DD for a date or YYYY-MM-DD HH:MM:SS for a timestamp.

df = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False,
                 infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=0)

C:\Users\jaros\AppData\Local\Temp\ipykernel_15868\2575660620.py:4: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False,
C:\Users\jaros\AppData\Local\Temp\ipykernel_15868\2575660620.py:4: UserWarning: Parsing dates in %d/%m/%Y %H:%M:%S format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False,


In [144]:
df.isna().sum()

Global_active_power          0
Global_reactive_power        0
Voltage                      0
Global_intensity             0
Sub_metering_1               0
Sub_metering_2               0
Sub_metering_3           25979
dtype: int64

In [145]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Global_active_power,2075259,4187,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_reactive_power,2075259,533,0.000,481561,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Voltage,2075259,2838,?,25979,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Global_intensity,2075259,222,1.000,172785,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_1,2075259,89,0.000,1880175,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_2,2075259,82,0.000,1436830,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sub_metering_3,2049280.0,NaN,NaN,NaN,6.458447,8.437154,0.0,0.0,1.0,17.0,31.0


In [146]:
df.isin(['?']).sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3               0
dtype: int64

In [147]:
df.replace('?', 'NaN', inplace=True)

In [148]:
df['Global_active_power'] = df['Global_active_power'].astype('float')
df['Global_reactive_power'] = df['Global_reactive_power'].astype('float')
df['Voltage'] = df['Voltage'].astype('float')
df['Global_intensity'] = df['Global_intensity'].astype('float')
df['Sub_metering_1'] = df['Sub_metering_1'].astype('float')
df['Sub_metering_2'] = df['Sub_metering_2'].astype('float')

In [149]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [150]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Global_active_power,2049280.0,1.091615,1.057294,0.076,0.308,0.602,1.528,11.122
Global_reactive_power,2049280.0,0.123714,0.112722,0.000,0.048,0.100,0.194,1.390
Voltage,2049280.0,240.839858,3.239987,223.200,238.990,241.010,242.890,254.150
Global_intensity,2049280.0,4.627759,4.444396,0.200,1.400,2.600,6.400,48.400
Sub_metering_1,2049280.0,1.121923,6.153031,0.000,0.000,0.000,0.000,88.000
Sub_metering_2,2049280.0,1.298520,5.822026,0.000,0.000,0.000,1.000,80.000
Sub_metering_3,2049280.0,6.458447,8.437154,0.000,0.000,1.000,17.000,31.000


In [151]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [ ]:
# Loop to fill missing values with a value at the same time one day ago
# (if the last value, will the same time one day before)

In [167]:
import os
from numba import jit, prange

os.environ["KMP_BLOCKTIME"] = "1"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"] = "granularity=fine,verbose,compact,1,0"
os.environ["OMP_NUM_THREADS"] = "10"

m,n = df.values.shape
@jit(parallel=True)
def fill_nan_parallel(df):
    for i in prange(m):
        for j in prange(n):
            if np.isnan(df.values[i,j]):
                df.values[i,j] = df.values[i-60*24,j]
fill_nan_parallel(df)


C:\Users\jaros\AppData\Local\Temp\ipykernel_15868\439127312.py:10: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "fill_nan_parallel" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\jaros\AppData\Local\Temp\ipykernel_15868\439127312.py (12)

File "..\..\..\..\AppData\Local\Temp\ipykernel_15868\439127312.py", line 12:
<source missing, REPL/exec in use?>

  @jit(parallel=True)
C:\Users\jaros\AppData\Local\Temp\ipykernel_15868\439127312.py:10: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "fill_nan_parallel" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "..\..\..\..\AppData\Local\Temp\ipykernel_15868\439127312.py", line 12:
<source missing, REPL/exec in use?>

  @jit(parallel=True)
c:\Anaconda3\lib\site-packages\numba\core\object_mode_passes.py:151: NumbaWarning: Function

In [ ]:
# Loop to fill missing values with a value at the same time one day ago
# (if the last value, will the same time one day before)

# for i in range(df.values.shape[0]):
#     for j in range(df.values.shape[1]):
#         if np.isnan(df.values[i, j]):
#             df.values[i, j] = df.values[i - 60 * 24, j]

In [ ]:
df.isna().sum()

In [ ]:
df.describe(include='all').T

In [ ]:
df.to_csv('house_energy_consumption.csv')

In [ ]:
df = pd.read_csv('house_energy_consumption.csv')

In [ ]:
df